In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from pyspark.ml.evaluation import BinaryClassificationEvaluator as BCE
from pyspark.ml.classification import LogisticRegression

import week4 as w4

## Lectura de los datos de la sesión anterior

En la sesión anterior terminamos guardando los datos en un fichero `.parquet`.
A continuación vamos a leerlo.

In [ ]:
df_raw = sqlContext.read.parquet('pca_features.parquet')

In [ ]:
df_raw.show(10)

Para poder aplicar el modelo de regressión logística, el campo que contiene la clase, debe ser de tipo `double` (número real), mientras que ahora lo tenemos de tipo `long` (número entero). Así que lo reformateamos.

In [ ]:
df = df_raw.withColumn('label', (df_raw['target'] - 1).astype('double'))['dr7objid', 'score', 'label']

In [ ]:
df.show(10)

## Modelo de Regresión Logística

En esta sección aplicaremos un algoritmo de regresión logística como el visto en el curso 3, para la clasificación de la imágenes de galaxias.

### Partición Test - Train

Dividiremos nuestro set de datos en dos subsets más pequeños
* df_train: para el entrenamiento del modelo (Train) con un 70% de los datos
* df_test: para la validación (Test) con un 30% de los datos

In [ ]:
(df_train, df_test) = df.randomSplit([0.7, 0.3], seed=123)

### Entrenamiento

In [ ]:
lgr = LogisticRegression(maxIter=100,
                         labelCol="label",
                         featuresCol="score",
                         tol=1e-18,
                         regParam=0.003)
                        
model = lgr.fit(df_train)

### Validación

Generamos la predicción según el modelo

In [ ]:
pred_df = model.transform(df_test)

Comparamos la predicción con la clasificación real

In [ ]:
cm = w4.confussion_matrix(pred_df, 'label', 'prediction').show()

Vemos que todos los indicadores de la bondad del modelo son bastante buenos. En particular podemos destacar que el porcentaje de aciertos (Accuracy) del modelo es del 83%.

También analizaremos la bondad del modelo a través de una curva ROC

In [ ]:
pred_df = pred_df.withColumn('probs', w4.ith_element(1)('probability'))

In [ ]:
w4.ROC_curve(pred_df)

Y vemos el área por debajo de la curva ROC. 

In [ ]:
print('AUC = ', BCE(metricName="areaUnderROC", rawPredictionCol='probability').evaluate(pred_df))

El valor de este area nos da una idea del poder de clasificación del modelo:
* ~1: clasificador perfecto
* ~0,5: clasificador aleatorio
* ~0: clasificador invertido